# Introduction : Modern Portfolio Theory

**Modern Portfolio Theory** est une théorie de l'investissement qui essaye de maximiser le rendement attendu sur la base d'un niveau de risque donné en choisissant minutieusement la proportions d'actifs dans le portefeuille.

Un portefeuille est un regroupement de différents types d'investissements tels que des actions, des obligations, des matières premières, des biens immobiliers, etc.

La théorie du portefeuille moderne suggère que le rendement attendu d'un investissement est directement lié au risque associé. Elle suggère qu'un investisseur doit choisir un portefeuille en fonction de son aversion au risque et de son horizon de placement.

Nous allons développer ici du code Python pour calculer les rendements attendus et la volatilité (risque) d'un portefeuille.

On utilise la bibliothèque pandas_datareader pour récupérer les données, numpy pour les opérations mathématiques, matplotlib pour visualiser et scipy pour l'optimisation.

Dans la suite de ce notebook, nous allons utiliser les notations suivantes :

- $R_p$ : rendement du portefeuille
- $w_i$ : proportion de l'actif $i$ dans le portefeuille
- $R_i$ : rendement de l'actif $i$
- $\Sigma_i$ : volatilité de l'actif $i$
- $\Sigma_p$ : volatilité du portefeuille
    - Les variables en majuscules sont des variables aléatoires
    - Les variables en minuscules sont deterministes
    - Les variables en gras sont des vecteurs.
----------

Notre objectif est de trouver les proportions $w_i$ qui maximisent le rendement du portefeuille pour un niveau de risque donné :

$$
\begin{align}
\max_{w_i} \quad & \mathbb{E}[R_p] \\
\text{sous contrainte} \quad & \mathbb{V}[R_p] < \sigma_{max}^2
\end{align}
$$

On définit le rendement du portefeuille comme la somme des rendements des actifs pondérés par leur proportion dans le portefeuille :

$$
R_p = \sum_i w_i R_i = \mathbf{w}^T \mathbf{R_p}
$$
Ainsi la variance du portefeuille est :

$$
\begin{align}
\mathbb{V}[R_p] &= \mathbb{V}[\mathbf{w}^T \mathbf{R_p}] \\
&= \mathbf{w}^T \mathbb{V}[\mathbf{R_p}] \mathbf{w} \\
&= \mathbf{w}^T \mathbf{\Sigma_p} \mathbf{w}
\end{align}
$$

Où $\mathbf{\Sigma_p}$ est la matrice de covariance des rendements des actifs du portefeuille. Pour rappel, la matrice de covariance est définie comme suit :

$$
\mathbf{\Sigma_p} = \begin{bmatrix}
\mathbb{V}[R_1] & \mathbb{C}[R_1, R_2] & \cdots & \mathbb{C}[R_1, R_n] \\
\mathbb{C}[R_2, R_1] & \mathbb{V}[R_2] & \cdots & \mathbb{C}[R_2, R_n] \\
\vdots & \vdots & \ddots & \vdots \\
\mathbb{C}[R_n, R_1] & \mathbb{C}[R_n, R_2] & \cdots & \mathbb{V}[R_n]
\end{bmatrix}
$$

Il nous reste donc à définir les $R_i$ et $\Sigma_i$ pour chaque actif du portefeuille de manière empirique. On notant $P_t$ le prix de l'actif $i$ à la date $t$, on définit le rendement de l'actif $i$ comme :

$$
R_i = \frac{P_t - P_{t-1}}{P_{t-1}} \text{ et } \Sigma_i = \sqrt{\mathbb{V}[R_i]}
$$



 

In [ ]:
#pip install numpy pandas matplotlib scipy pandas_datareader plotly nbformat

In [2]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader as web
import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
pio.templates.default = "plotly_dark"

## Etape 1 : Récupération des données

Dans cette étape, nous allons récupérer les données des actifs qui composent notre portfolio. Pour cet exemple, nous allons considérer 3 actions : Google ('GOOG'), Apple ('AAPL') et Microsoft ('MSFT'). Mais vous pouvez ajouter autant d'actions que vous le souhaitez.

In [3]:
def get_data(assets, start_date, end_date):
    """
    Récupère les données des actions sur Yahoo Finance
    """
    return yf.download(assets, start=start_date, end=end_date)['Adj Close']

In [4]:
# Liste des actifs dans le portefeuille
assets = ['AAPL', 'AMZN', 'MSFT', 'GOOG', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
N = 365
start_date = dt.datetime.now() - dt.timedelta(days=N)
end_date = dt.datetime.now()

data = get_data(assets, start_date, end_date)

[*********************100%%**********************]  9 of 9 completed


In [5]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

Comme nous avons vu dans le cours, pour faire de l'optimisation de portefeuille, nous avons besoin des rendements historiques des actifs. Nous allons donc récupérer les données sur une période de 5 ans.

## Etape 2 : Calcul des returns

Dans cette étape, nous allons calculer les rendements quotidiens des actions. Les rendements sont calculés en utilisant la formule suivante :
$$
R_t = \frac{P_{t} - P_{t-1}}{P_{t-1}}
$$
Où $P_{t}$ est le prix à la date $t$ et $P_{t-1}$ est le prix à la date $t-1$.
En fait, nous calculons le pourcentage de changement de prix entre deux jours consécutifs, c'est pour cela que l'on divise par $P_{t-1}$.

In [6]:
def compute_portfolio_return(data):
    return data.pct_change()

In [7]:
returns = compute_portfolio_return(data)
# Affichage des rendements journaliers
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=returns.index, y=returns[a], name=a))
fig.update_layout(title='Rendements journaliers des actifs', yaxis_title='Rendements')
fig.show()

Ici nous avons accès aux réalisations des rendements $R_i(\omega)$ et nous voulons estimer les paramètres de la distribution des rendements ($\mu$ et $\sigma$). 

In [8]:
# Returns distribution
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Histogram(x=returns[a], name=a))
fig.update_layout(title='Distribution des rendements journaliers', yaxis_title='Fréquence')
fig.show()

Nous avons accès à $\mathbb{E}[R_i]$ et $\mathbb{V}[R_i]$. Stockons ces valeurs dans un dataframe.

In [9]:
# stock expected return and volatility in one dataframe
returns_mean = returns.mean()
returns_std = returns.std()
returns_mean_std = pd.concat([returns_mean, returns_std], axis=1)
returns_mean_std.columns = ['Mean', 'Std']
returns_mean_std

,Mean,Std
AAPL,0.001351,0.013653
ADBE,0.002620,0.020006
AMZN,0.002172,0.021332
GOOG,0.001745,0.019666
MSFT,0.001788,0.016255
NFLX,0.001802,0.024621
NVDA,0.004479,0.031559
PYPL,-0.000682,0.023068
TSLA,0.002002,0.036276


## Etape 3 : Matrice de covariance

Maintenant que nous avons les rendements historiques des actifs, nous pouvons calculer la matrice de covariance des rendements. La matrice de covariance est définie comme suit :

$$
\mathbf{\Sigma_p} = \begin{bmatrix}
\mathbb{V}[R_1] & \mathbb{C}[R_1, R_2] & \cdots & \mathbb{C}[R_1, R_n] \\
\mathbb{C}[R_2, R_1] & \mathbb{V}[R_2] & \cdots & \mathbb{C}[R_2, R_n] \\
\vdots & \vdots & \ddots & \vdots \\
\mathbb{C}[R_n, R_1] & \mathbb{C}[R_n, R_2] & \cdots & \mathbb{V}[R_n]
\end{bmatrix}
$$

La covariance empirique entre deux actifs $i$ et $j$ est définie comme suit :

$$
\mathbb{C}[R_i, R_j] = \frac{1}{T-1} \sum_{t=1}^T (R_i(t) - \mathbb{E}[R_i])(R_j(t) - \mathbb{E}[R_j])
$$


In [10]:
# Calcul de la matrice de correlation
corr_matrix = returns.corr()
cov_matrix = returns.cov()
# Affichage de la matrice de correlation with x axis and y axis with the same size
fig = go.Figure(data=go.Heatmap(z=corr_matrix, x=assets, y=assets, colorscale='Darkmint'))
fig.update_layout(
    margin=dict(l=350, r=350, t=50, b=50),
)
fig.update_layout(title='Matrice de corrélation', yaxis_title='Actifs', xaxis_title='Actifs')
fig.show()



On peut maintenant essayer de visualiser calculer la valeur de notre portefeuille pour différentes valeurs de $w_i$.

In [18]:
# On va donc créer une fonction qui prend en entrée un vecteur de poids et qui retourne la valeur du portefeuille.
def compute_portfolio_value(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return np.dot(returns.mean(), weights) * len(returns)

def compute_portfolio_risk(weights, returns):
    """
    weights: array of weights
    returns: dataframe of risk
    """
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights))) * np.sqrt(len(returns))

w_test = np.array([1/len(assets)]*len(assets))
portfolio_return = compute_portfolio_value(w_test, returns)

initial_portfolio_value = 100

print('Valeur du portefeuille: ', np.round((1+portfolio_return) * (initial_portfolio_value)), '$')
print('Risque du portefeuille: ', np.round(compute_portfolio_risk(np.array([1/len(assets)]*len(assets)), returns), 4))

Valeur du portefeuille:  148.0 $
Risque du portefeuille:  0.2517


# Etape 4 : Optimisation du portefeuille

Maintenant que nous avons toutes les informations nécessaires, nous pouvons commencer à optimiser notre portefeuille. 

$$
\begin{align}
\max_{w_i} \quad & \mathbf{w}^T \mathbf{R_p} \\
\text{sous contrainte} \quad & \mathbf{w}^T \mathbf{\Sigma_p} \mathbf{w} < \sigma_{max}^2
\end{align}
$$

Utilisons Monte Carlo pour générer des valeurs aléatoires de $w_i$ et calculons le rendement et la volatilité du portefeuille pour chaque valeur de $w_i$.

In [20]:

def generate_random_weights(n):
    """
    n: number of weights to generate
    """
    weights = np.random.rand(n)
    return weights / sum(weights)

def generate_random_portfolio(returns):
    """
    returns: dataframe of returns
    """
    weights = generate_random_weights(len(returns.columns))
    
    portfolio_return = compute_portfolio_value(weights, returns)
    portfolio_risk = compute_portfolio_risk(weights, returns)

    return weights, portfolio_return, portfolio_risk


In [25]:

# On va générer 10000 portefeuilles aléatoires et les afficher sur un graphique.
N = 10000
SIGMA_MAX = 0.016

random_portfolios = np.zeros((3, N))
best_weights = np.zeros(len(assets))
best_portfolio_return = 0
for i in range(N):
    weights, portfolio_return, portfolio_risk = generate_random_portfolio(returns)
    random_portfolios[0, i] = portfolio_return
    random_portfolios[1, i] = portfolio_risk
    random_portfolios[2, i] = (portfolio_return / portfolio_risk)
    if portfolio_return > best_portfolio_return and portfolio_risk < SIGMA_MAX:
        best_portfolio_return = portfolio_return
        best_weights = weights

# ---------------------------- Affichage des portefeuilles aléatoires ----------------------------
fig = go.Figure()
fig.update_layout(
    margin=dict(l=200, r=200, t=50, b=50),
)
fig.add_trace(go.Scatter(x=random_portfolios[1], y=random_portfolios[0], mode='markers', marker=dict(color=random_portfolios[2], colorscale='RdBu', showscale=False), text=assets, name='Portefeuilles aléatoires'))

fig.update_layout(title='Frontière efficiente', yaxis_title='Rendement', xaxis_title='Risque')

fig.add_trace(go.Scatter(x=[random_portfolios[1][np.argmax(random_portfolios[2])]], y=[random_portfolios[0][np.argmax(random_portfolios[2])]], mode='markers', marker=dict(color='green', size=10), name='Max Sharp Ratio', marker_symbol='star'))
fig.add_trace(go.Scatter(x=[random_portfolios[1][np.argmin(random_portfolios[1])]], y=[random_portfolios[0][np.argmin(random_portfolios[1])]], mode='markers', marker=dict(color='red', size=10), name='Min Volatility'))
fig.show()
# -------------------------------------------------------------------------------------------------




In [146]:
portfolio_return = compute_portfolio_value(best_weights, returns)
portfolio_risk = np.sqrt(np.dot(best_weights.T, np.dot(returns.cov(), best_weights)))
print('Valeur du portefeuille: ', np.round((1+portfolio_return) * (initial_portfolio_value)), '$')

Valeur du portefeuille:  158.0 $


In [14]:
# Affichage de la fonction frontière efficiente comme etant la courbe interpolée des portefeuilles aléatoires de la frontière efficiente
def compute_portfolio_risk(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov(), weights)))
def compute_portfolio_return(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return np.dot(returns.mean(), weights) * len(returns)
def compute_portfolio_sharp_ratio(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return compute_portfolio_return(weights, returns) / compute_portfolio_risk(weights, returns)
def compute_portfolio(weights, returns):
    """
    weights: array of weights
    returns: dataframe of returns
    """
    return np.array([compute_portfolio_return(weights, returns), compute_portfolio_risk(weights, returns), compute_portfolio_sharp_ratio(weights, returns)])




# Etape 5 : Test d'une stratégie d'investissement